# [SOLUTION] Exercise - Build a Web-Aware Agent with Search and Knowledge Comparison

In this exercise, you'll build an agent that can search the web for current information and compare
it with its internal knowledge. This demonstrates how to enhance an LLM's capabilities with real-time
web data and how to critically analyze differences between sources.


## Challenge

Your task is to create an agent that can:

- Implement web search functionality using Tavily API
- Parse and process search results effectively
- Handle different types of queries (news, facts, events)
- Extract relevant information from search results

## Setup
First, let's import the necessary libraries:

In [1]:
import os
from datetime import datetime
from typing import List, Dict
from dotenv import load_dotenv
from tavily import TavilyClient

from lib.agents import Agent
from lib.messages import BaseMessage
from lib.tooling import tool

In [2]:
load_dotenv()

True

## Play with Tavily

In [3]:
api_key = os.getenv("TAVILY_API_KEY")
client = TavilyClient(api_key=api_key)

In [4]:
result = client.search("What's Nintendo?")

In [5]:
result

{'query': "What's Nintendo?",
 'follow_up_questions': None,
 'answer': None,
 'images': [],
 'results': [{'url': 'https://en.wikipedia.org/wiki/Nintendo',
   'title': 'Nintendo - Wikipedia',
   'content': '**Nintendo Co., Ltd.** is a Japanese multinational video game company headquartered in Kyoto. The history of Nintendo began when craftsman Fusajiro Yamauchi founded the company to produce handmade *hanafuda* playing cards. The company became internationally dominant in the 1980s after the arcade release of *Donkey Kong "Donkey Kong (1981 video game)")* (1981) and the Nintendo Entertainment System, which launched outside of Japan alongside *Super Mario Bros.* in 1985. Nintendo was founded as Nintendo Koppai on 23 September 1889 by craftsman Fusajiro Yamauchi in Shimogyō-ku, Kyoto, Japan, as an unincorporated establishment, to produce and distribute Japanese playing cards, or karuta (かるた; from Portuguese *carta*, \'card\'), most notably *hanafuda* (花札, \'flower cards\'). His first acti

## Define Web Search tool

In [6]:
@tool
def web_search(query: str, search_depth: str = "advanced") -> Dict:
    """
    Search the web using Tavily API
    args:
        query (str): Search query
        search_depth (str): Type of search - 'basic' or 'advanced' (default: advanced)
    """
    api_key = os.getenv("TAVILY_API_KEY")
    client = TavilyClient(api_key=api_key)
    
    # Perform the search
    search_result = client.search(
        query=query,
        search_depth=search_depth,
        include_answer=True,
        include_raw_content=False,
        include_images=False
    )
    
    # Format the results
    formatted_results = {
        "answer": search_result.get("answer", ""),
        "results": search_result.get("results", []),
        "search_metadata": {
            "timestamp": datetime.now().isoformat(),
            "query": query
        }
    }
    
    return formatted_results

In [7]:
tools = [web_search]

In [8]:
simple_agent = Agent(
    model_name="gpt-4o-mini",
    instructions=("You are a helpful assistant"),
)

In [9]:
web_agent = Agent(
    model_name="gpt-4o-mini",
    instructions=(
            "You are a web-aware assistant that can search for update information "
            "For each query, you will search the web for current information using " 
            "Tavily's AI-optimized search and provide a comprehensive answer\n"
            "Always cite your sources and explain any discrepancies found.\n"
            "Be particularly attentive to dates and time-sensitive information."
    ),
    tools=tools
)

In [10]:
def print_messages(messages: List[BaseMessage]):
    for m in messages:
        print(f" -> (role = {m.role}, content = {m.content}, tool_calls = {getattr(m, 'tool_calls', None)})")

## Run your Agents

**Simple Agent**

**Note**: This example relies on the date being recent enough that the answer will not be in the model's training data. Try with other current events/dates if needed to get similar results.

In [11]:
run1 = simple_agent.invoke(
    query="Who won the 2025 Oscar for International Movie?", 
)

print("\nMessages from run 1:")
messages = run1.get_final_state()["messages"]
print_messages(messages)

[StateMachine] Starting: __entry__
[StateMachine] Executing step: message_prep
[StateMachine] Executing step: llm_processor
[StateMachine] Terminating: __termination__

Messages from run 1:
 -> (role = system, content = You are a helpful assistant, tool_calls = None)
 -> (role = user, content = Who won the 2025 Oscar for International Movie?, tool_calls = None)
 -> (role = assistant, content = I'm sorry, but I don't have access to information beyond October 2021, and I cannot provide details about events or awards that occurred after that date, including the 2025 Oscars. You may want to check an up-to-date source for the latest information on the Oscars., tool_calls = None)


In [12]:
print(run1.get_final_state()["messages"][-1].content)

I'm sorry, but I don't have access to information beyond October 2021, and I cannot provide details about events or awards that occurred after that date, including the 2025 Oscars. You may want to check an up-to-date source for the latest information on the Oscars.


In [13]:
run2 = simple_agent.invoke(
    query="What are the most recent developments in AI technology?", 
)
print("\nMessages from run 2:")
messages = run2.get_final_state()["messages"]
print_messages(messages)

[StateMachine] Starting: __entry__
[StateMachine] Executing step: message_prep
[StateMachine] Executing step: llm_processor
[StateMachine] Terminating: __termination__

Messages from run 2:
 -> (role = system, content = You are a helpful assistant, tool_calls = None)
 -> (role = user, content = Who won the 2025 Oscar for International Movie?, tool_calls = None)
 -> (role = assistant, content = I'm sorry, but I don't have access to information beyond October 2021, and I cannot provide details about events or awards that occurred after that date, including the 2025 Oscars. You may want to check an up-to-date source for the latest information on the Oscars., tool_calls = None)
 -> (role = user, content = What are the most recent developments in AI technology?, tool_calls = None)
 -> (role = assistant, content = As of my last update in October 2021, I can't provide real-time developments or advancements in AI technology that may have occurred after that date. However, I can summarize some 

In [14]:
print(run2.get_final_state()["messages"][-1].content)

As of my last update in October 2021, I can't provide real-time developments or advancements in AI technology that may have occurred after that date. However, I can summarize some trends and areas of research that were prominent up to that time:

1. **Natural Language Processing (NLP)**: Advances in transformer-based models, such as OpenAI's GPT-3, have greatly improved the capabilities of AI in understanding and generating human language. Research continues into making these models more efficient and capable of reasoning.

2. **AI Ethics and Fairness**: There has been increasing focus on the ethical implications of AI, including biases in algorithms and the need for fairness and transparency in AI systems. Organizations and researchers are working on frameworks and guidelines to address these issues.

3. **AI in Healthcare**: AI applications in healthcare have expanded, including diagnostic tools powered by machine learning, personalized medicine, and drug discovery. The COVID-19 pand

**Web Agent**

In [15]:
run1 = web_agent.invoke(
    query="Who won the 2025 Oscar for International Movie?", 
)

print("\nMessages from run 1:")
messages = run1.get_final_state()["messages"]
print_messages(messages)

[StateMachine] Starting: __entry__
[StateMachine] Executing step: message_prep
[StateMachine] Executing step: llm_processor
[StateMachine] Executing step: tool_executor
[StateMachine] Executing step: llm_processor
[StateMachine] Terminating: __termination__

Messages from run 1:
 -> (role = system, content = You are a web-aware assistant that can search for update information For each query, you will search the web for current information using Tavily's AI-optimized search and provide a comprehensive answer
Always cite your sources and explain any discrepancies found.
Be particularly attentive to dates and time-sensitive information., tool_calls = None)
 -> (role = user, content = Who won the 2025 Oscar for International Movie?, tool_calls = None)
 -> (role = assistant, content = None, tool_calls = [ChatCompletionMessageToolCall(id='call_fEjY9phxA0T1F5qEN5HskyVm', function=Function(arguments='{"query":"2025 Oscar for International Movie winner"}', name='web_search'), type='function')])

In [16]:
print(run1.get_final_state()["messages"][-1].content)

The winner of the 2025 Oscar for Best International Feature Film was Brazil's *I'm Still Here*. This marked Brazil's first win in this category. The film was directed by Walter Salles, who dedicated the award to Eunice Pavia, a real-life figure recognized for her resistance during Brazil's military dictatorship in the 1970s.

The Oscar ceremony took place on March 2, 2025, where *I'm Still Here* triumphed over notable competitors, including France's *Emilia Pérez*, Denmark's *The Girl with the Needle*, Germany's *The Seed of the Sacred Fig*, and Latvia's *Flow* (which won the award for Best Animated Feature) (Sources: [Hollywood Reporter](https://www.hollywoodreporter.com/lists/oscars-2025-winners-list/), [People](https://people.com/oscars-2025-im-still-here-brazil-best-international-feature-11684447), [NPR](https://www.npr.org/2025/03/02/nx-s1-5315313/oscars-2025-im-still-here-brazil-best-international-feature)).


In [17]:
run2 = web_agent.invoke(
    query="What are the most recent developments in AI technology?", 
)
print("\nMessages from run 2:")
messages = run2.get_final_state()["messages"]
print_messages(messages)

[StateMachine] Starting: __entry__
[StateMachine] Executing step: message_prep
[StateMachine] Executing step: llm_processor
[StateMachine] Executing step: tool_executor
[StateMachine] Executing step: llm_processor
[StateMachine] Terminating: __termination__

Messages from run 2:
 -> (role = system, content = You are a web-aware assistant that can search for update information For each query, you will search the web for current information using Tavily's AI-optimized search and provide a comprehensive answer
Always cite your sources and explain any discrepancies found.
Be particularly attentive to dates and time-sensitive information., tool_calls = None)
 -> (role = user, content = Who won the 2025 Oscar for International Movie?, tool_calls = None)
 -> (role = assistant, content = None, tool_calls = [ChatCompletionMessageToolCall(id='call_fEjY9phxA0T1F5qEN5HskyVm', function=Function(arguments='{"query":"2025 Oscar for International Movie winner"}', name='web_search'), type='function')])

In [18]:
print(run2.get_final_state()["messages"][-1].content)

As of 2025, several significant developments in AI technology have emerged, reflecting the rapid evolution of the field. Here are some key trends and advancements:

1. **Generative AI Expansion**: Generative AI has become deeply integrated across various industries, enabling the creation of original content, such as text, images, music, and code. Major advancements have been powered by large language models (LLMs), facilitating applications ranging from drafting emails to synthesizing medical images (Source: [Nucamp](https://www.nucamp.co/blog/ai-essentials-for-work-2025-top-10-ai-trends-professionals-need-to-watch-in-2025)).

2. **Agentic AI Systems**: The concept of agentic AI—autonomous systems capable of collaboration with humans—has gained traction. Companies like Google are reimagining software development by moving beyond mere coding assistance to creating powerful systems that interact with developers (Source: [Google](https://blog.google/technology/ai/2025-research-breakthroug

## Advanced

You can modify `agents.py` to include: 
- a comparison field in the state schema
- a web search step
- a comparison step in the workflow